## 房屋資料抓取

In [4]:
import requests
headers = {
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'   
}
res = requests.get('https://sale.591.com.tw/home/search/list?type=2&&shType=list&section=7&regionid=1&kind=9&firstRow=30&totalRows=2404&timestamp=1539754751297', headers = headers)

In [5]:
res

<Response [200]>

In [11]:
import pandas
pandas.DataFrame(res.json()['data']['house_list']).head(3)

,address,area,browsenum,cartmodel,carttype,community_link,community_name,delivery,down_price_percent,fci_pai,...,saletype,section_name,shape_name,showhouseage,showprice,tag,title,type,unit_price,unitprice
0,忠孝東路五段31巷,15.16,382,,,/newCommunity-index.html?cid=50519,無爭,,NaN,,...,0,信義區,電梯大樓,10年,"1,790",[],市府極美無爭精品,2,118.07萬/坪,118.07
1,忠孝東路五段743巷,32.56,606,,,/newCommunity-index.html?cid=93550,達麗信義,,NaN,,...,0,信義區,電梯大樓,3年,"2,600","[含車位, 有陽台]",指名承辦人小鍾：本棟最低總價！精緻裝潢！,2,114.77萬/坪,114.77
2,松山路,20.68,720,,,/newCommunity-index.html?cid=6751,永春大樓,,NaN,,...,0,信義區,電梯大樓,34年,"1,220",[有陽台],專售永春大樓-住商住匠團隊,2,58.99萬/坪,58.99


In [14]:
import time
houseurl = 'https://sale.591.com.tw/home/search/list?type=2&&shType=list&section=7&regionid=1&kind=9&firstRow={}&totalRows=2404&timestamp=1539754751297'

houseary = []
for i in range(0,int(2404 / 30)+ 1 ):
    res = requests.get(houseurl.format(i*30), headers = headers)
    df = pandas.DataFrame(res.json()['data']['house_list'])
    houseary.append(df)
    time.sleep(1)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [15]:
housedf = pandas.concat(houseary)

In [17]:
housedf.count()

address                 2403
area                    2405
browsenum               2405
build_purpose              2
call_num                   2
cartmodel               2403
carttype                2403
community_link          2403
community_name          2403
company                    2
delivery                2403
down_price_percent        35
fci_pai                 2403
floor                   2403
has_carport             2403
houseage                2405
houseid                 2405
is_carport              2403
is_combine              2403
is_down_price           2403
is_full                    2
is_hurry_price          2403
is_newhouse                2
is_oversea              2403
is_pro_advertisement       2
is_video                2403
isnew                   2403
isvip                   2403
kind                    2403
kind_name               2403
label                      2
mainarea                2403
nick_name               2403
original_price            35
phonenum      

In [18]:
housedf.to_excel('house591.xlsx')

## 房屋資料清理

In [47]:
import pandas
housedf  = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/chinalife/master/data/house591.xlsx')

In [48]:
housedf.iloc[2]

address                                                            忠孝東路五段
area                                                                41.17
browsenum                                                             180
build_purpose                                                         NaN
call_num                                                              NaN
cartmodel                                                             NaN
carttype                                                              NaN
community_link                          /newCommunity-index.html?cid=6305
community_name                                                       春秋大廈
company                                                               NaN
delivery                                                              NaN
down_price_percent                                                    NaN
fci_pai                                                               NaN
floor                                 

In [49]:
housedf = housedf[['area', 'room','floor','price','houseage']]

In [50]:
housedf[['bedroom', 'livingroom', 'bathroom']] = housedf['room'].str.extract('(\d+)房(\d+)廳(\d+)衛')

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [51]:
housedf[['current_floor', 'total_floor']] = housedf['floor'].str.extract('(\d+)F/(\d+)F')

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [52]:
del housedf['room']
del housedf['floor']

In [53]:
housedf.dropna(inplace=True)

In [54]:
housedf.head()

,area,price,houseage,bedroom,livingroom,bathroom,current_floor,total_floor
2,41.17,3988,35,4,2,2,8,14
3,15.76,1180,44,2,1,1,3,4
6,106.74,11500,4,4,2,3,7,12
7,15.39,1520,19,1,1,1,4,14
8,53.39,3850,29,2,2,2,8,15


In [55]:
housedf.columns

Index(['area', 'price', 'houseage', 'bedroom', 'livingroom', 'bathroom',
       'current_floor', 'total_floor'],
      dtype='object')

In [59]:
Y = housedf['price'].values
X = housedf[['area', 'bedroom', 'livingroom', 'bathroom', 'current_floor',
       'total_floor', 'houseage']]

In [60]:
X['total_floor'] = X['total_floor'].astype(float)
X['bedroom'] = X['bedroom'].astype(float)
X['livingroom'] = X['livingroom'].astype(float)
X['bathroom'] = X['bathroom'].astype(float)
X['current_floor'] = X['current_floor'].astype(float)

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

## 資料分析

In [61]:
import statsmodels.api as sm
X2  = sm.add_constant(X)
est = sm.OLS(Y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.890
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                     2502.
Date:                Wed, 17 Oct 2018   Prob (F-statistic):               0.00
Time:                        14:18:00   Log-Likelihood:                -18696.
No. Observations:                2177   AIC:                         3.741e+04
Df Residuals:                    2169   BIC:                         3.745e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -1096.2896    137.992     -7.945